In [2]:
!python --version

Python 3.10.11


In [44]:
%load_ext autoreload
%autoreload 2

# UMM Discovery

## Import libraries

In [45]:
# Check for GPU device
import os
print(os.getenv('CUDA_VISIBLE_DEVICES'))

None


In [46]:
from numba.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning, NumbaPerformanceWarning
import warnings

warnings.filterwarnings(action='once')
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)
warnings.simplefilter("ignore", category=PendingDeprecationWarning)

In [47]:
pip install joblib==1.1.0

/opt/conda/envs/umm_discovery_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Note: you may need to restart the kernel to use updated packages.


In [49]:
import os
import shutil
import time
import pandas as pd
import numpy as np

from sklearn.metrics.cluster import normalized_mutual_info_score

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

import matplotlib.pyplot as plt

import clustering
from model import MultiScaleNet
from util import Logger, UnifLabelSampler
from training import compute_features, train
from correction import do_batch_correction
from evaluation import evaluate_epoch, evaluate_training
from my_dataset import bbbc021_dataset
from plot import plot_training, plot_training_summary, plot_inter_part_validation, plot_inter_hier_validation, plot_external_validation, plot_NSC_NSB, plot_total_score

## Set parameter

In [13]:
class args:
    # Experiment ID
    run_ID = "EXP01"
    
    # path to dataset
    data = "/UMM-Discovery/data"
    
    # file name of the metadata csv file
    csv = "UMM_meta.csv"
    
    # path to the output folder
    save_path = "/UMM-Discovery/out"
    output_path = "" #dont assign

    # CNN architecture 
    arch = "msnet" 

    # input channels
    n_input_dim = 3
    
    # number of features
    n_features = 64
    
    # Batch correction ["TVN", "COMBAT", MNN]
    batch_corrections = ["COMBAT", "TVN"] 
    
    # dimension reduction method (default: PCA)
    # 'PCA' or 'UMAP' or 'TSNE' or 'AdaptiveTSNE'
    dim_method = 'PCA' 
    
    # dimensions after reduction
    n_components = 16
    
    # clustering algorithm (default: Kmeans)
    # 'Kmeans' or 'AdaptiveKmeans' or 'PIC' or 'HDBscan' 
    clustering = 'Kmeans'

    # number of cluster for k-means (default: 10000)
    nmb_cluster = 104
    
    # parameter for adaptive k-means
    end_k = 39
    end_epochs = 150
    
    # parameter for PIC
    sigma = 0.2 
    nnn = 5
    
    # paramter for HDBscan
    min_cluster_size = 12
    min_samples = 3
    nnn = 25
    
    # mini-batch size (default: 256)
    batch = 16
    
    # learning rate (default: 0.05)
    lr = 0.05
    
    # weight decay pow (default: -5)
    wd=-5
    
    # number of total epochs to run (default: 200)
    epochs = 50

    # momentum (default: 0.9)
    momentum = 0.9

    # how many epochs of training between two consecutive reassignments of clusters (default: 1)
    reassign = 1

    # number of data loading workers (default: 4)
    workers=4
    
    # cuda version
    cuda = 10
    
    # Epoch to continue from (default: None)
    resume = None
    
    # manual epoch number (useful on restarts) (default: 0)
    start_epoch = 0
    
    # random seed (default: 42)
    seed = 12
    
    # chatty
    verbose = True

In [3]:
exp_name = ""
exp_name +='{}_'.format(args.run_ID)

## Add clustering algorithm
if args.clustering == "Kmeans":
    clustering_parameter = 'Kmeans_cl{}'.format(args.nmb_cluster)
elif args.clustering == "AdaptiveKmeans":
    clustering_parameter = 'AdaptiveKmeans_cl{}-{}_ep{}'.format(args.nmb_cluster, args.end_k, args.end_epochs)
elif args.clustering == "PIC":
    clustering_parameter = 'PIC_sig{}_nn{}'.format(args.sigma, args.nnn)
elif args.clustering == "HDBscan":
    clustering_parameter = 'HDBscan_mincl{}-minsam{}_nn{}'.format(args.min_cluster_size, args.min_samples, args.nnn)

exp_name +='{}'.format(clustering_parameter)
    
## Add dimensionality reduction method
exp_name +='_{}-{}d'.format(args.dim_method, args.n_components)

## Add batch correction
batch_corrections_parameter = None
if args.batch_corrections == None:
    batch_corrections_parameter = "NoCorrection"
else:
    for corr in args.batch_corrections:
        if batch_corrections_parameter is None:
            batch_corrections_parameter = corr
        else:
            batch_corrections_parameter +='_{}'.format(corr)
exp_name +='_{}'.format(batch_corrections_parameter)
    
print('Experiment: {}'.format(exp_name))

Experiment: EXP01_Kmeans_cl104_PCA-16d_COMBAT_TVN


In [4]:
args.output_path = os.path.join(args.save_path, exp_name)
print('Output path: {}'.format(args.output_path))

Output path: /path/to/output/folder/EXP01_Kmeans_cl104_PCA-16d_COMBAT_TVN


In [5]:
embed_path = os.path.join(args.output_path, 'y-embed_{}.txt'.format(exp_name))
metadata_path = os.path.join(args.output_path, 'metadata_{}.txt'.format(exp_name))
assign_epoch_path = os.path.join(args.output_path, 'epoch_assignment_{}.txt'.format(exp_name))
metrices_path = os.path.join(args.output_path, 'metrices_{}.txt'.format(exp_name))
loss_path = os.path.join(args.output_path, 'loss_{}.txt'.format(exp_name))

best_labeled_sil_embed_path = os.path.join(args.output_path, 'best_labeled_sil_y-embed_{}.txt'.format(exp_name))
best_labeled_nsc_embed_path = os.path.join(args.output_path, 'best_labeled_nsc_y-embed_{}.txt'.format(exp_name))
best_unlabeled_embed_path = os.path.join(args.output_path, 'best_unlabeled_y-embed_{}.txt'.format(exp_name))

In [6]:
embeds_cols = ['Z{:03d}'.format(i) for i in range(args.n_features)]

## Create Neural Network

In [9]:
# fix random seeds
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)

# CNN
if args.verbose:
    print('Create Multi-Scale Neural Network')
    
model = MultiScaleNet(input_dim=3, num_features=args.n_features, num_classes=args.nmb_cluster)
fd = int(model.top_layer.weight.size()[1])
model.top_layer = None
model.features = torch.nn.DataParallel(model.features)
model.cuda()
cudnn.benchmark = True

# create optimizer
optimizer = torch.optim.SGD(
    filter(lambda x: x.requires_grad, model.parameters()),
    lr=args.lr,
    momentum=args.momentum,
    weight_decay=10**args.wd,
)

# define loss function
criterion = nn.CrossEntropyLoss().cuda()

# optionally resume from a checkpoint
if args.resume:
    checkpoint_path = os.path.join(args.output_path, 'checkpoints', 'checkpoint_epoch{}.pth.tar'.format(args.resume))
    if os.path.isfile(checkpoint_path):
        print("=> loading checkpoint '{}'".format(checkpoint_path))
        checkpoint = torch.load(checkpoint_path)
        args.start_epoch = checkpoint['epoch']
        # remove top_layer parameters from checkpoint
        for key in list(checkpoint['state_dict']):
            if 'top_layer' in key:
                del checkpoint['state_dict'][key]
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

# creating checkpoint repo
exp_check = os.path.join(args.output_path, 'checkpoints')
if not os.path.isdir(exp_check):
    os.makedirs(exp_check)

# creating cluster assignments log
cluster_log = Logger(os.path.join(args.output_path, 'train_data.pkl'))
if args.resume:
    cluster_log.load_data(args.resume)

Create Multi-Scale Neural Network


In [10]:
if args.verbose:
    summary(model, input_size=(3, 512, 640))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         MaxPool2d-1          [-1, 3, 256, 320]               0
            Conv2d-2          [-1, 6, 512, 640]             168
       BatchNorm2d-3          [-1, 6, 512, 640]              12
              ReLU-4          [-1, 6, 512, 640]               0
            Conv2d-5          [-1, 9, 512, 640]             495
       BatchNorm2d-6          [-1, 9, 512, 640]              18
              ReLU-7          [-1, 9, 512, 640]               0
            Conv2d-8          [-1, 9, 512, 640]              90
       BatchNorm2d-9          [-1, 9, 512, 640]              18
             ReLU-10          [-1, 9, 512, 640]               0
        MaxPool2d-11          [-1, 9, 256, 320]               0
           Conv2d-12         [-1, 12, 256, 320]             156
      BatchNorm2d-13         [-1, 12, 256, 320]              24
             ReLU-14         [-1, 12, 2

In [11]:
if args.cuda==10:
    # create tensorboard writer
    tensorboard_path = os.path.join(args.save_path, 'runs', exp_name)

    if os.path.isdir(tensorboard_path) and not args.resume:
        shutil.rmtree(tensorboard_path)

    if not os.path.isdir(tensorboard_path):
        os.makedirs(tensorboard_path)
    
    from torch.utils.tensorboard import SummaryWriter
    tensorboard_writer = SummaryWriter(log_dir=tensorboard_path)
else:
    tensorboard_writer = None

## Load the data

In [61]:
root_dir = args.data
metadata = args.csv
shabi = pd.read_csv(os.path.join(root_dir, metadata))

In [59]:
shabi = pd.read_csv("UMM_meta.csv")

In [36]:
for k in shabi.keys():
    shabi[k]
    break

In [62]:
shabi

,table_nr,ImageNumber,filename_dna,plate,filename_tubulin,Image_PathName_Tubulin,filename_actin,Image_PathName_Actin,Image_Metadata_Plate_DAPI,Image_Metadata_Well_DAPI,Replicate,compound,compound_uM,MoA,Unique_MoA,Unique_Treatments,Unique_Compounds
0,1,9,Week1_150607_B04_s1_w11323931B-BDA7-4F42-870E-...,Week1_22123,Week1_150607_B04_s1_w2F8F7EA7A-EC57-49CA-A556-...,Week1/Week1_22123,Week1_150607_B04_s1_w494DCA5C4-3531-497D-A8B0-...,Week1/Week1_22123,Week1_22123,B04,1,cytochalasin B,10.0,Actin disruptors,0,0,0
1,1,10,Week1_150607_B04_s2_w1F649C703-6FA8-406F-8575-...,Week1_22123,Week1_150607_B04_s2_w2802CC81E-56F9-41C5-A6C2-...,Week1/Week1_22123,Week1_150607_B04_s2_w4342F300D-60F8-4256-A637-...,Week1/Week1_22123,Week1_22123,B04,1,cytochalasin B,10.0,Actin disruptors,0,0,0
2,1,11,Week1_150607_B04_s3_w135D66B4C-0548-4AB8-A57B-...,Week1_22123,Week1_150607_B04_s3_w2D81AEFB9-9DD4-4B59-9177-...,Week1/Week1_22123,Week1_150607_B04_s3_w408BE006A-BF34-457E-81A9-...,Week1/Week1_22123,Week1_22123,B04,1,cytochalasin B,10.0,Actin disruptors,0,0,0
3,1,12,Week1_150607_B04_s4_w1EB720AD1-83BA-48A7-8C47-...,Week1_22123,Week1_150607_B04_s4_w261B79A05-7534-46F3-8C80-...,Week1/Week1_22123,Week1_150607_B04_s4_w49FF7E7B1-F049-4994-BCA2-...,Week1/Week1_22123,Week1_22123,B04,1,cytochalasin B,10.0,Actin disruptors,0,0,0
4,1,249,Week1_150607_B04_s1_w17D5A011C-EA14-4EA7-A62C-...,Week1_22141,Week1_150607_B04_s1_w20ED4F2D9-0D4A-4CA6-A13D-...,Week1/Week1_22141,Week1_150607_B04_s1_w413EE84DD-212B-4008-99A9-...,Week1/Week1_22141,Week1_22141,B04,2,cytochalasin B,10.0,Actin disruptors,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3843,9,3560,Week9_090907_F11_s4_w19580FF4D-DC3D-4BD0-93FE-...,Week9_39301,Week9_090907_F11_s4_w2DC65EC6F-BDCA-4B05-B243-...,Week9/Week9_39301,Week9_090907_F11_s4_w45699A0F4-9AEE-4CD4-8973-...,Week9/Week9_39301,Week9_39301,F11,3,DMSO,0.0,DMSO,0,0,0
3844,9,3597,Week9_090907_G11_s1_w1EDE534D2-FCEE-4F92-A30B-...,Week9_39301,Week9_090907_G11_s1_w26A22E27F-6A81-43F5-9587-...,Week9/Week9_39301,Week9_090907_G11_s1_w4554A2BF7-0D53-4D27-BF92-...,Week9/Week9_39301,Week9_39301,G11,3,DMSO,0.0,DMSO,0,0,0
3845,9,3598,Week9_090907_G11_s2_w10B010F39-3B4B-4DCB-8E34-...,Week9_39301,Week9_090907_G11_s2_w2720AC778-3F85-4293-8D75-...,Week9/Week9_39301,Week9_090907_G11_s2_w49B290958-BCF2-4DDD-B0E9-...,Week9/Week9_39301,Week9_39301,G11,3,DMSO,0.0,DMSO,0,0,0
3846,9,3599,Week9_090907_G11_s3_w10394282C-6D3D-4E0E-9FA3-...,Week9_39301,Week9_090907_G11_s3_w24C59DB62-E99B-4284-BAD2-...,Week9/Week9_39301,Week9_090907_G11_s3_w471FE25C8-2477-456F-9D74-...,Week9/Week9_39301,Week9_39301,G11,3,DMSO,0.0,DMSO,0,0,0


In [66]:
tra = [transforms.ToTensor()]
    
# load the data
end = time.time()

dataset = bbbc021_dataset(args.data, args.csv, 'MoA', transform=transforms.Compose(tra))

if args.verbose: 
    print('Load dataset: {0:.2f} s'.format(time.time() - end))
    
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=args.batch,
                                         num_workers=args.workers,
                                         pin_memory=True)

Load dataset: 3.77 s


### Check data

In [ ]:
dataset.get_df().head()

In [ ]:
print('Number of compounds: {}'.format(len(dataset.get_df()['compound'].unique())))
print('Number of treatments: {}'.format(len(dataset.get_df()['pseudoclass'].unique())))

In [ ]:
dataset.get_df().shape

In [ ]:
for i in range(3):
    img = dataset[i][0]

    plt.figure(figsize=(20,10))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img[0,:,:])
    
    ax = plt.subplot(1, 3, 2)
    plt.imshow(img[1,:,:])
    
    ax = plt.subplot(1, 3, 3)
    plt.imshow(img[2,:,:])

    plt.show()

## Setup clustering algorithm

In [ ]:
# clustering algorithm to use
if args.clustering == "Kmeans":
    deepcluster = clustering.Kmeans(k=args.nmb_cluster, 
                                    dim_method=args.dim_method, 
                                    n_components=args.n_components,
                                    n_jobs=args.workers+1)
elif args.clustering == "AdaptiveKmeans":
    deepcluster = clustering.AdaptiveKmeans(initial_k=args.nmb_cluster, 
                                            end_k=args.end_k, 
                                            end_epochs=args.end_epochs, 
                                            initial_epoch=args.start_epoch, 
                                            dim_method=args.dim_method, 
                                            n_components=args.n_components,
                                            n_jobs=args.workers+1)
elif args.clustering == "PIC":
    deepcluster = clustering.PIC(sigma=args.sigma, 
                                 nnn=args.nnn, 
                                 dim_method=args.dim_method, 
                                 n_components=args.n_components,
                                 n_jobs=args.workers+1)
elif args.clustering == "HDBscan":
    deepcluster = clustering.HDBscan(min_cluster_size=args.min_cluster_size, 
                                    min_samples=args.min_samples, 
                                    nnn=args.nnn, 
                                    dim_method=args.dim_method, 
                                    n_components=args.n_components,
                                    n_jobs=args.workers+1)

## Train neural network

In [ ]:
loss_list = []
nmi_list = [0]
best_NSC_acc = 0
df_meta = dataset.get_df()

In [ ]:
# training convnet with DeepCluster
start_train_time = time.time()
for epoch in range(args.start_epoch, args.epochs):
    end = time.time()

    # remove head
    model.top_layer = None
    model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])

    # compute features for the whole dataset
    features = compute_features(dataloader, model, len(dataset), args)
    #print(features.shape)
    embeds = pd.DataFrame(features, columns=embeds_cols)
    embeds = pd.concat([df_meta, embeds], axis=1)
    
    # batch correction 
    embeds = do_batch_correction(embeds, embeds_cols, args.batch_corrections, verbose=args.verbose)
        
    # evaluate features
    df_eval = evaluate_epoch(embeds.copy(), embeds_cols, verbose=args.verbose)
    
    # best epoch
    NSC_acc = df_eval["NSC_1-NN_treatment"][0]
    if NSC_acc > best_NSC_acc:
        best_NSC_acc = NSC_acc
    
    # cluster the features
    clustering_loss = deepcluster.cluster(embeds[embeds_cols].to_numpy(copy=True), verbose=args.verbose)

    # save cluster assignment
    df_assign_epoch = pd.DataFrame(clustering.arrange_clustering(deepcluster.images_lists), columns=['Assignment_{}'.format(epoch)])

    # assign pseudo-labels
    train_dataset = clustering.cluster_assign(deepcluster.images_lists, dataset.imgs)

    # uniformely sample per target
    sampler = UnifLabelSampler(int(args.reassign * len(train_dataset)),
                               deepcluster.images_lists)

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch,
        num_workers=args.workers,
        sampler=sampler,
        pin_memory=True,
    )

    # set last fully connected layer
    mlp = list(model.classifier.children())
    mlp.append(nn.ReLU(inplace=True).cuda())
    model.classifier = nn.Sequential(*mlp)
    model.top_layer = nn.Linear(args.n_features, len(deepcluster.images_lists))
    model.top_layer.weight.data.normal_(0, 0.01)
    model.top_layer.bias.data.zero_()
    model.top_layer.cuda()

    # train network with clusters as pseudo-labels
    start_epoch_train_time = time.time()
    loss = train(train_dataloader, model, criterion, optimizer, epoch, args)
    loss_list.append(float(loss))

    # print log
    if args.verbose:
        print('###### Epoch [{0}] ###### \n'
              'Total Time: {1:.3f} s\n'
              'Training time: {2:.3f} s\n'
              'ConvNet loss: {3:.3f} \n'
              'NSC acc: {4:.3f} \n'
              'Silhoutte score: {5:.3f} \n'
              'Best NSC acc: {6:.3f}'
              .format(epoch, 
                      time.time() - end, 
                      time.time() - start_epoch_train_time, 
                      loss, 
                      df_eval["NSC_1-NN_treatment"][0]*100, 
                      df_eval["silhou_moa"][0],
                      best_NSC_acc*100
                     ))
        try:
            if epoch > 0:
                nmi = normalized_mutual_info_score(
                    clustering.arrange_clustering(deepcluster.images_lists),
                    clustering.arrange_clustering(cluster_log.data['clustering'][-1])
                )
                print('NMI against previous assignment: {0:.3f}'.format(nmi))
            else:
                nmi = 0
            nmi_list.append(nmi)
        except IndexError:
            pass
        print('####################### \n')
    # save running checkpoint
    torch.save({'epoch': epoch + 1,
                'arch': args.arch,
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict()},
               os.path.join(args.output_path, 'checkpoint.pth.tar'))

    # save cluster assignments, evaluation data and features
    cluster_log.log(epoch, embeds, df_eval, df_assign_epoch, loss, nmi, deepcluster.images_lists)
    
    # write to tensorboard
    if tensorboard_writer is not None:
        tensorboard_writer.add_scalar('loss', loss, epoch)
        tensorboard_writer.add_scalar('nmi', nmi, epoch)
        for eval_metric, data in df_eval.iteritems():
            tensorboard_writer.add_scalar(eval_metric, data[0], epoch)
        
print('Total training time {}'.format(time.time() - start_train_time))

# Evaluate training

In [ ]:
cluster_log.data.keys()

In [ ]:
cluster_log.data['features'][0]

In [ ]:
df_metric = pd.concat(cluster_log.data['eval_metrices'], axis=0)
df_metric["epoch"] = cluster_log.data['epoch']
df_metric["loss"] = cluster_log.data['loss']
df_metric["loss"] = df_metric["loss"].map(float)
df_metric["nmi"] = cluster_log.data['nmi']
df_metric["nmi"] = df_metric["nmi"].map(float)
df_metric = df_metric.reset_index(drop=True)

In [ ]:
df_metric = df_metric.rename(columns={'n_clusters_well':'n_clusters'})
df_metric['total_score'] = (df_metric['adj-mut_comp_pred'] + df_metric['comple_treat-pred'] + df_metric['silhou_pred_tsne'])/3

In [ ]:
df_metric.head()

In [ ]:
plot_training(df_metric, args.output_path)

## Overall summary

In [ ]:
plot_training_summary(df_metric, args.output_path)

### Internal partitional validation

In [ ]:
plot_inter_part_validation(df_metric, args.output_path)

### Internal hierarchical validation

In [ ]:
plot_inter_hier_validation(df_metric, args.output_path)

### External Validation

In [ ]:
plot_external_validation(df_metric, args.output_path)

In [ ]:
plot_NSC_NSB(df_metric, args.output_path)

### Total score

In [ ]:
plot_total_score(df_metric, args.output_path)

## Find best epoch for labeled and unlabeled situation

### Best epoch labeled

In [ ]:
best_epoch_labeled_silh = df_metric['silhou_moa_tsne'].idxmax()
print('Best epoch (Silhouette): ' , best_epoch_labeled_silh)
print('NSC: ', df_metric["NSC_1-NN_treatment"][best_epoch_labeled_silh])
print('Silhouette moa tsne: ', df_metric['silhou_moa_tsne'][best_epoch_labeled_silh])

In [ ]:
best_epoch_labeled_nsc = df_metric['NSC_1-NN_treatment'].idxmax()
print('Best epoch (NSC): ' , best_epoch_labeled_nsc)
print('NSC: ', df_metric["NSC_1-NN_treatment"][best_epoch_labeled_nsc])
print('Silhouette moa tsne: ', df_metric['silhou_moa_tsne'][best_epoch_labeled_nsc])

### Best epoch unlabeled

In [ ]:
best_epoch_unlabeled = df_metric.loc[:,'total_score'].idxmax()
print('Best epoch (unlabeled): ' , best_epoch_unlabeled)
print('NSC: ', df_metric['NSC_1-NN_treatment'][best_epoch_unlabeled])
print('Silhouette pred tsne: ', df_metric['silhou_pred_tsne'][best_epoch_unlabeled])
print('Adjusted Rand index (treat-pred): ', df_metric['adj-rand_treat-pred'][best_epoch_unlabeled])
print('Adjusted Mutual information (treat-pred): ', df_metric['adj-mut_treat-pred'][best_epoch_unlabeled])
print('Adjusted Mutual information (moa-pred): ', df_metric['adj-mut_moa-pred'][best_epoch_unlabeled])
print('Jaccard (treat-pred): ', df_metric['jaccard_treat-pred'][best_epoch_unlabeled])
print('Completeness (treat-pred): ', df_metric['comple_treat-pred'][best_epoch_unlabeled])
print('Number of clusters: ', df_metric['n_clusters'][best_epoch_unlabeled])
print('Total score: ', df_metric['total_score'][best_epoch_unlabeled])

# Save files

#### Save hyperparameter with best values

In [ ]:
if tensorboard_writer is not None:
    tensorboard_writer.add_hparams(hparam_dict={
                                    'cuda_version': args.cuda,
                                    'lr': args.lr, 
                                    'batch_size': args.batch, 
                                    'n_features': args.n_features,
                                    'batch_correction': batch_corrections_parameter,
                                    'dim_method': args.dim_method, 
                                    'n_features_reduced': args.n_components,
                                    'clustering_algorithmn': args.clustering,
                                    'clustering_parameter': clustering_parameter,
        
                                   }, 
                                   metric_dict={
                                       # labeled best silhoutte score moa
                                       'labeled_best_silh/epoch': best_epoch_labeled_silh,
                                       'labeled_best_silh/silhou_moa_tsne': df_metric['silhou_moa_tsne'][best_epoch_labeled_silh],
                                       'labeled_best_silh/NSC_1-NN_treatment': df_metric['NSC_1-NN_treatment'][best_epoch_labeled_silh],
                                       'labeled_best_silh/NSC_1-NN_avg': df_metric['NSC_1-NN_avg'][best_epoch_labeled_silh],
                                       # labeled NSC
                                       'labeled_best_nsc/epoch': best_epoch_labeled_nsc,
                                       'labeled_best_nsc/silhou_moa_tsne': df_metric['silhou_moa_tsne'][best_epoch_labeled_nsc],
                                       'labeled_best_nsc/NSC_1-NN_treatment': df_metric['NSC_1-NN_treatment'][best_epoch_labeled_nsc],
                                       'labeled_best_nsc/NSC_1-NN_avg': df_metric['NSC_1-NN_avg'][best_epoch_labeled_nsc],
                                       # unlabeled
                                       'unlabeled/epoch': best_epoch_unlabeled,
                                       'unlabeled/NSC_1-NN_treatment': df_metric['NSC_1-NN_treatment'][best_epoch_unlabeled],
                                       'unlabeled/silhou_moa_tsne': df_metric['silhou_moa_tsne'][best_epoch_unlabeled],
                                       'unlabeled/silhou_pred_tsne': df_metric['silhou_pred_tsne'][best_epoch_unlabeled],
                                       'unlabeled/adj-rand_treat-pred': df_metric['adj-rand_treat-pred'][best_epoch_unlabeled],
                                       'unlabeled/adj-mut_treat-pred': df_metric['adj-mut_treat-pred'][best_epoch_unlabeled],
                                       'unlabeled/jaccard_treat-pred': df_metric['jaccard_treat-pred'][best_epoch_unlabeled],
                                       'unlabeled/comple_treat-pred': df_metric['comple_treat-pred'][best_epoch_unlabeled],
                                       'unlabeled/n_clusters': df_metric['n_clusters'][best_epoch_unlabeled],
                                       'unlabeled/total_score': df_metric['total_score'][best_epoch_unlabeled]
                                   })

In [ ]:
tensorboard_writer.close()

#### Save metrices

In [ ]:
df_metric.to_csv(metrices_path, sep='\t', index=False)

#### Save metadata

In [ ]:
df_meta = dataset.get_df()
df_meta.to_csv(metadata_path, sep='\t', index=False)

#### Save cluster assignments

In [ ]:
assign_list = cluster_log.data['cluster_assignments']
df_assignments = pd.concat(assign_list, axis=1)
df_assignments = pd.concat([df_meta, df_assignments], axis=1)
df_assignments.to_csv(assign_epoch_path, sep='\t', index=False)

#### Save loss

In [ ]:
loss_df = pd.DataFrame({'model_loss':cluster_log.data['loss'], 
                        'NMI':cluster_log.data['nmi']})
loss_df.to_csv(loss_path, index=False, sep='\t')

#### Save the best embedding

In [ ]:
df_best_labeled_sil_embeds = cluster_log.data['features'][best_epoch_labeled_silh]
df_best_labeled_nsc_embeds = cluster_log.data['features'][best_epoch_labeled_nsc]
df_best_unlabeled_embeds = cluster_log.data['features'][best_epoch_unlabeled]

df_best_labeled_sil_embeds.to_csv(best_labeled_sil_embed_path, sep='\t', index=False)
df_best_labeled_nsc_embeds.to_csv(best_labeled_nsc_embed_path, sep='\t', index=False)
df_best_unlabeled_embeds.to_csv(best_unlabeled_embed_path, sep='\t', index=False)

## Best epoch evaluation

In [ ]:
df_best_labeled_sil_save_well = evaluate_training(df_best_labeled_sil_embeds, embeds_cols, os.path.join(args.output_path,'best_labeled_sil'), verbose=args.verbose)

In [ ]:
df_best_labeled_ncs_save_well = evaluate_training(df_best_labeled_nsc_embeds, embeds_cols, os.path.join(args.output_path,'best_labeled_nsc'), verbose=args.verbose)

In [ ]:
df_best_unlabeled_save_well = evaluate_training(df_best_unlabeled_embeds, embeds_cols, os.path.join(args.output_path,'best_unlabeled'), verbose=args.verbose)

In [ ]:
print("finished")